# データの永続化
## SQLite
* 軽量データベースを実装するC言語のライブラリ
* サーバー用のプロセスを必要としない, 組み込み型のデータベースエンジン.
* 単一のファイルによって管理される（インメモリも可能）

In [2]:
import sqlite3

In [3]:
# DBファイルの保存先パス（相対パスで指定）
path = ''

# Google Colabの場合
# path = '/content/'

# DBファイル
db_name = 'test.db'

# DB接続の確率
# DBファイルが存在しない場合は新規作成される
conn = sqlite3.connect(path + db_name)

# DBへの接続を閉じる
conn.close()

## こういうデータベースを作成してみる
### car.db
| id | name | price |
| - |- | - |
|  |  |   |
|  |  |   |
|  |  |   |

### DB(SQlite)を使う時の処理の流れ
1. DBコネクションを確立
2. コネクションオブジェクトからデータ操作用のカーソルオブジェクトを取得
3. 実行したいSQLを定義
4. SQLを実行
5. 必要があればコミットする
6. コネクションを閉じる

## テーブルを作成

In [22]:
path = ''
db_name = 'car.db'

try:
    # DBオブジェクトの作成
    conn = sqlite3.connect(path + db_name)

    # SQL(RDBを操作するための言語)を実行するためのカーソルオブジェクトを取得
    cur = conn.cursor()
    # SQL文の作成
    # テーブルの作成
    sql = 'CREATE TABLE cars (id INT, name TEXT, price REAL);'

    # SQL文の実行
    cur.execute(sql)

    # 変更をDBに反映させる
    conn.commit()

except sqlite3.Error as e:
    print('エラーが発生しました:', e)
    
finally:
    # DBへの接続を閉じる
    conn.close()


### データをDBに挿入する（単一レコード）

In [23]:

path = ''
db_name = 'car.db'

try:
    # DB接続オブジェクトの作成
    conn = sqlite3.connect(path + db_name)

    # SQL(RDBを操作するための言語)を実行するためのカーソルオブジェクトを取得
    cur = conn.cursor()
    # SQL文の作成
    # データの挿入
    # INSERT INTO テーブル名　VALUES (列に対応したデータをカンマ区切りで);
    # 文字列データを挿入する場合は、文字列をシングルクォーテーションで挟む
    sql = "INSERT INTO cars (id, name, price) VALUES (1, '370z', 7000000);"

    # SQL文の実行
    cur.execute(sql)

    # 変更をDBに反映させる
    conn.commit()
except sqlite3.Error as e:
    print('エラーが発生しました:', e)
finally:
    # DBへの接続を閉じる
    conn.close()

### データを挿入する（複数レコード）

In [24]:
path = ''
db_name = 'car.db'

try:
    # DB接続オブジェクトの作成
    conn = sqlite3.connect(path + db_name)

    # SQL(RDBを操作するための言語)を実行するためのカーソルオブジェクトを取得
    cur = conn.cursor()
    # SQL文の作成
    # 複数レコードの挿入
    # INSERT INTO テーブル名　VALUES (列に対応したプレースホルダーをカンマ区切りで);
    sql = "INSERT INTO cars (id, name, price) VALUES (?, ?, ?);"

    cars = [
        (2, '911カレラ', 15500000),
        (3, 'GRスープラ', 8000000),
        (4, 'シビック Type-R', 5000000),
    ]

    # SQL文の実行
    cur.executemany(sql, cars)

    # 変更をDBに反映させる
    conn.commit()
except sqlite3.Error as e:
    print('エラーが発生しました:', e)
finally:
    # DBへの接続を閉じる
    conn.close()

### DB内のデータを参照する

In [25]:

path = ''
db_name = 'car.db'

try:
    # DB接続オブジェクトの作成
    conn = sqlite3.connect(path + db_name)

    # SQL(RDBを操作するための言語)を実行するためのカーソルオブジェクトを取得
    cur = conn.cursor()
    
    # データを取得するSQL
    # SELECT * FROM テーブル名;
    # *の部分は、取得したい列の名前を、区切りで指定することもできる。
    sql = "SELECT * FROM cars;"

    # SQL文の実行
    cur.execute(sql)

    # 変更をDBに反映させる
    conn.commit()

except sqlite3.Error as e:
    print('エラーが発生しました:', e)

else:
    for row in cur:
        # 行データ(row)はタプルなので、アンパックして列データを取得
        id, name, price = row
        print(id, name, price)

finally:
    # DBへの接続を閉じる
    conn.close()

1 370z 7000000.0
2 911カレラ 15500000.0
3 GRスープラ 8000000.0
4 シビック Type-R 5000000.0


### データの削除

In [27]:

path = ''
db_name = 'car.db'

try:
    # DB接続オブジェクトの作成
    conn = sqlite3.connect(path + db_name)

    # SQL(RDBを操作するための言語)を実行するためのカーソルオブジェクトを取得
    cur = conn.cursor()
    
    # データを削除するSQL
    # DELETE FROM テーブル名 WHERE id = 1;
    sql = "DELETE FROM cars WHERE id = 1;"
    # sql = "DELETE FROM cars WHERE name = '911カレラ';"
    sql = "DELETE FROM cars WHERE name Like '%カレラ%';"

    # SQL文の実行
    cur.execute(sql)

    # 変更をDBに反映させる
    conn.commit()

except sqlite3.Error as e:
    print('エラーが発生しました:', e)

finally:
    # DBへの接続を閉じる
    conn.close()

### データの更新

In [28]:

path = ''
db_name = 'car.db'

try:
    # DB接続オブジェクトの作成
    conn = sqlite3.connect(path + db_name)

    # SQL(RDBを操作するための言語)を実行するためのカーソルオブジェクトを取得
    cur = conn.cursor()
    
    # データを更新するSQL
    # UPDATE テーブル名　SET　カラム名=新しい値 WHERE 更新したい行を指定する条件;
    sql_update = "UPDATE cars SET price = ? WHERE id = ?;"
    
    # SQL文の実行
    cur.execute(sql_update, (5000000, 2))

    # 変更をDBに反映させる
    conn.commit()

except sqlite3.Error as e:
    print('エラーが発生しました:', e)

finally:
    # DBへの接続を閉じる
    conn.close()